<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.03/bds_pipeline_006_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Usando variables numéricas y categóricas juntas</font>**

❤ https://scikit-learn.org/stable/

<p align="justify">
👀 En los Colabs anteriores, mostramos el preprocesamiento para variables numéricas y variables categóricas. Sin embargo, ese proceso lo realizamos para tratar cada tipo de variable individualmente, es decir, por un lado las variables numéricas y por otro, las variables categóricas. En este Colab, vamos a mostrar cómo combinar estos pasos de preprocesamiento.
<br><br>
Primero cargamos todo el conjunto de datos del censo de adultos.
</p>



In [1]:
import numpy as np
import pandas as pd

In [2]:
adult_census = pd.read_csv("https://raw.githubusercontent.com/cristiandarioortegayubro/BDS/main/datasets/adult_census.csv")

In [3]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name])

In [4]:
data.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States


 # **<font color="DeepPink">Selección basada en tipos de datos</font>**

<p align="justify">
👀 Separaremos variables categóricas y numéricas usando sus tipos de datos para identificarlas, ya que vimos anteriormente que objeto corresponde a las columnas categóricas (cadenas de caracteres). Hacemos uso del <code>make_column_selector</code> para seleccionar las columnas correspondientes.
</p>


In [5]:
from sklearn.compose import make_column_selector as selector

<p align="justify">
👀 En el selector de las columnas numericas excluimos los tipos de datos <code>object</code> porque podemos tener numeros enteros o numeros decimales.
</p>


In [6]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

In [7]:
numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

In [8]:
numerical_columns

['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [9]:
categorical_columns

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

 # **<font color="DeepPink">Enviar columnas a un procesador específico</font>**

<p align="justify">
👀 Anteriormente vimos que debemos tratar los datos de manera diferente dependiendo de su naturaleza, es decir, si los datos son numéricos o si los datos son categóricos.
<br><br>
<code>scikit-learn</code> proporciona una clase denominada <code>ColumnTransformer</code> que enviará información específica de las columnas a un transformador específico, lo que facilita el ajuste del modelo predictivo en un conjunto de datos que combina ambos tipos de variables, las numéricas y las categóricas, concluyendo en datos tabulares tipificados heterogéneamente.
<br><br>Entonces, primero definimos las columnas dependiendo de su tipo de datos:
</p>


* Se aplicará `one-hot-encoding` a las columnas categóricas. Además, usaremos `handle_unknown="ignore"` para resolver problemas potenciales debido a categorías.
* Se aplicará el escalado numérico para características numéricas que serán estandarizadas.

<p align="justify">
👀 Ahora, creamos nuestro <code>ColumnTransformer</code> especificando tres valores:</p> 

1. el nombre del preprocesador, 
1. el transformador y 
1. las columnas.

<p align="justify">
👀 Ahora vamos a crear los preprocesadores para los datos numéricos y los datos categóricos.



In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [11]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

<p align="justify">
👀 Ahora, creamos el transformador y asociamos cada uno de estos preprocesadores con sus respectivas columnas.
</p>

https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

In [12]:
from sklearn.compose import ColumnTransformer

In [13]:
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/ColumnTransformers-001.png?raw=true" width="600">
</p>


`ColumnTransformer` hace lo siguiente:

* Divide las columnas del conjunto de datos original según los nombres de las columnas o los índices proporcionados. Obtendremos tantos subconjuntos como transformadores pasen al ColumnTransformer.
* Transforma cada subconjunto. Se aplica un transformador específico a cada subconjunto: llamará internamente a` fit_transform` o `transform`. El resultado de este paso es un conjunto de conjuntos de datos transformados.
* Luego concatena los conjuntos de datos transformados en un único conjunto de datos.

<p align="justify">
Lo importante es que <code>ColumnTransformer</code> es como cualquier otro transformador de <code>scikit-learn</code>. En particular, se puede combinar con un clasificador en un Pipeline:
</p>

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [15]:
model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['age', 'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

El modelo final es más complejo que los modelos anteriores:

* Se llama al método <code>fit</code> para preprocesar los datos y luego entrenar al clasificador de los datos preprocesados.
* El método <code>predict</code> hace predicciones sobre los nuevos datos.
* El método <code>score</code> se utiliza para predecir los datos de prueba y comparar las predicciones con las etiquetas de prueba esperadas para calcular la precisión.

<br>
<p align="justify">
Comencemos por dividir nuestros datos en el conjunto de datos de entrenamiento y en el conjunto de datos de prueba.

 # **<font color="DeepPink">Train-test, división del conjunto de datos</font>**

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
data_train, data_test, target_train, target_test = train_test_split(data, 
                                                                    target, 
                                                                    random_state=42)

<p align="justify">
👀 Tenga en cuenta que usamos <code>train_test_split</code> con fines didácticos, para mostrar <code>scikit-learn</code>. En un entorno real, uno podría preferir usar la validación cruzada para poder evaluar también la incertidumbre de nuestra estimación del rendimiento de un modelo, como se demostró anteriormente.
</p>

In [18]:
_ = model.fit(data_train, target_train)

 # **<font color="DeepPink">Ajuste y prediccion</font>**

<p align="justify">
👀 Luego, podemos enviar el conjunto de datos sin procesar directamente al Pipeline. De hecho, no necesitamos realizar ningún preprocesamiento manual, ni llamar a los métodos <code>transform</code> o <code>fit_transform</code>, ya que se resolverá al llamar al método <code>predict</code>. 
<br><br>
Como ejemplo, predecimos sobre las $10$ primeras muestras del conjunto de datos de prueba...
</p>

In [19]:
data_test.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
7762,56,Private,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,0,40,United-States
23881,25,Private,HS-grad,9,Married-civ-spouse,Transport-moving,Own-child,Other,Male,0,0,40,United-States
30507,43,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,14344,0,40,United-States
28911,32,Private,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
19484,39,Private,Bachelors,13,Married-civ-spouse,Sales,Wife,White,Female,0,0,30,United-States


In [20]:
model.predict(data_test)[:10]

array([' <=50K', ' <=50K', ' >50K', ' <=50K', ' >50K', ' <=50K', ' <=50K',
       ' >50K', ' <=50K', ' <=50K'], dtype=object)

In [21]:
target_test[:10]

7762      <=50K
23881     <=50K
30507      >50K
28911     <=50K
19484     <=50K
43031     <=50K
28188     <=50K
12761      >50K
40834     <=50K
27875     <=50K
Name: class, dtype: object

<p align="justify">
👀 Para obtener directamente la métrica <code>Accuracy</code>, necesitamos llamar los métodos <code>score</code>. Esto computa el puntaje de <code>Accuracy</code> en el conjunto de datos de prueba.
</p>

In [22]:
model.score(data_test, target_test).round(4)

0.8578

 # **<font color="DeepPink">Evaluación del modelo con Cross-validation</font>**

<p align="justify">
👀 Un modelo predictivo puede ser evaluado con validación cruzada....
</p>


In [23]:
from sklearn.model_selection import cross_validate

In [24]:
cv_results = cross_validate(model, data, target, cv=5)
cv_results

{'fit_time': array([0.78920293, 0.8688972 , 0.79974484, 1.14087629, 0.96567678]),
 'score_time': array([0.03224802, 0.03203106, 0.03234601, 0.05209565, 0.03287172]),
 'test_score': array([0.85116184, 0.8498311 , 0.84756347, 0.85268223, 0.85513923])}

In [25]:
scores = cv_results["test_score"]
print("")
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")


The mean cross-validation accuracy is: 0.851 ± 0.003


<p align="justify">
👀 El modelo compuesto tiene una mayor precisión predictiva que los dos modelos que utilizan variables numéricas y categóricas de forma aislada.
</p>


 # **<font color="DeepPink">Adaptando un modelo más potente</font>**

<p align="justify">
👀 Los modelos lineales son buenos porque suelen ser mas simples para entrenar,
pequeños para implementar y rápidos para predecir.
<br><br> 
Sin embargo, a menudo es útil comprobar si los modelos más complejos, como un conjunto de árboles de decisión puede conducir a un mayor rendimiento predictivo. Usaremos un modelo llamado <code> gradient-boosting trees</code> y podemos evaluar su rendimiento. 
<br><br>
Más precisamente, el modelo de <code>scikit-learn</code> que usaremos se llama <code>HistGradientBoostingClassifier</code>. Tenga en cuenta que
estos tipos de modelos, los modelos boosting, se desarrollarán en otros Colabs.
<br><br> 
Para modelos basados en árboles, el manejo de variables numéricas y categóricas es más simple que para los modelos lineales...
</p>


* No necesitamos escalar las características numéricas.
* Usan una codificación ordinal para las variables categóricas.


<p align="justify">
👀 Por lo tanto, para <code>HistGradientBoostingClassifier</code>, el Pipeline de preprocesamiento es un poco más simple que el que vimos antes para <code>LogisticRegression</code>:
</p>


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html

In [26]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder

In [27]:
categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

In [28]:
preprocessor = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_columns)],
    remainder="passthrough")

In [29]:
model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

In [30]:
%%time
_ = model.fit(data_train, target_train)

CPU times: user 1.43 s, sys: 21.5 ms, total: 1.45 s
Wall time: 799 ms


In [31]:
model.score(data_test, target_test).round(4)

0.8808

<p align="justify">
👀 Podemos observar que obtenemos precisiones significativamente más altas con el modelo <code>Gradient Boosting</code>. Esto es a menudo lo que observamos cuando el conjunto de datos tiene una gran cantidad de muestras y una cantidad limitada de características, por ejemplo menos de 1000 con una combinación de variables numéricas y categóricas.
<br><br>
😀 Esto explica por qué las <code>Gradient Boosted Machines</code> son muy populares entre los Científicos de Datos que trabajan con datos tabulares.
</p>


 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:<br>
<br>
✅ Cargamos los datos de un archivo <code>CSV</code> usando <code>Pandas</code>.
<br>
✅ Se usó un <code>ColumnTransformer</code> para  variables categóricas y numéricas.
<br>
✅ Se usó un Pipeline para encadenar el preprocesamiento de <code>ColumnTransformer</code>. 
<br>
✅ Se vio que hay modelos que pueden superar a los modelos lineales.
</p>




<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
